# Full pipeline

1. Trailmap <br/>
    1.1 Training <br/>
    1.2 Output <br/>
2. Skeletons <br/>
    2.1 Skeletonize_3d <br/>
    2.2 Removal of small, disconnected objects
3. Corrections and Atlas alignment
4. Plots and Results
5. Appendix

---
### Mice:
Mesospim data: <br/>
Al207
Al209
Al215
Al211
Al210
Al213
Al246
Al249

---
### Packages:
It is best to create two separate environments. <br/> 
1. Training and segmentation (tensorflow)
2. Data analysis, Elastix, image processing, cython, else...

#### Env1:
**Version 1** (default, not recommended): <br/>
> conda create -n trailmap_env tensorflow-gpu=2.1 opencv=3.4.2 pillow=7.0.0 python=3.7 <br/>

(Installs automatically cuda-toolkit, cudnn from anaconda for env only)

**Version 2** (recommended): <br/>
> conda create -n trailmap_env python=3.9 <br/>
> conda activate trailmap_env <br/>
> pip install tensorflow <br/>
> pip install pillow <br/>
> pip install opencv-python <br/>

**You need to install** latest nvidia-driver for the gfx card, cuda-toolkit and cudnn from nvidia website. **/!\ guidelines** ! <br/>
To keep in mind: 
- cuda-toolkit needs to be compatible with the gfx card. cudnn needs to be compatible with cuda-toolkit.
- Tensorflow version has to be compatible with cuda-toolkit, cudnn and python-version.
- For the RTX3090, cudnn should be in version 8.1+ if possible (because of hardware architecture), otherwise you will get some substantial loss in performance. 
- Tensorflow is recommended to be at least in version 2.4 for RTX 30 series GPUs.
- pip installed *tensorflow=2.8*. Need to check if all packages are compatible but should be good. (Use standard checks: gpu hardware detection, ...)
- pip is not monitored by anaconda (no check for compatibility and no logs in env versions)

#### Env2:
> conda create -n data_analysis python=3.9

- Would not mess with python=3.10 at the moment.
- Would recommend to use *mamba* to install some of the packages (faster and less annoying than conda). Has to be installed in (base) environment. 
> conda deactivate <br/>
> conda install -c conda-forge mamba <br/>
> conda activate data_analysis <br/>
> mamba install tqdm scikit-image ... <br/>
> pip install matplotlib <br/>
> pip install SimpleITK <br/>
- Addionnal package: tqdm, matplotlib, SimpleITK, skimage, plotly, dask-image, cython, (+whatever else you need)
- You can use **conda list** (or **mamba list**) on terminal to get version of all installed packages within environment.

In [1]:
# For data_analysis env

import os
import tkinter.filedialog as fdialog #window to select directories
from tqdm import tqdm #progress bars

import shutil
import time

#import anne_code.Neuron_analysis as na #custum functions
#from anne_code.Neuron_analysis import *

import re
import numpy as np
import numpy.ma as ma
import pandas as pd
import matplotlib.pyplot as plt

import SimpleITK as sitk #elastix
#import skimage
from skimage import io, morphology, measure
import cv2
from PIL import Image

import warnings
import plotly.graph_objects as go
import cc3d

from project_module import *

In [2]:
%load_ext autoreload
%autoreload 2

## 1. Trailmap

See Trailmap README on how to train and use the current weights of the model to get the probabilitic maps. <br/>
Below some utility function: the images have to be within the same folder. Might want to extract a list of images from a .tiff stack.

In [5]:
image_stack = fdialog.askopenfilename(title='Please select tiff stack')
output_folder = fdialog.askdirectory(title='Please select the output directory')
#mice_names = ['AL207','AL209','AL215','AL211','AL210','AL213','AL246','AL249']
#data_folder_lsens = '/home/lucasdelez/Documents/lsens_server/data/' #data en mounted lsens access drive
#image_stack = data_folder_lsens + mice_names[0]
#output_folder = '/home/lucasdelez/Documents/segmented_images/Al207_1'

print("Selected mice:", image_stack)
print("Folder name to copy:",'\n\t',output_folder)

Selected mice: /data/Lucas/AL210/AL210_rLS.tif
Folder name to copy: 
	 /data/Lucas/AL210/Al210_1


In [85]:
# Convert image stack to folder containing individual images.
image_stack = r'/data/Lucas/AL215/AL215_rLS.tif'
output_folder = r'/data/Lucas/AL215/Al215_1'
mice_name = r'AL215_rLS'
from_stack_to_folder(image_stack, output_folder, mice_name, '.tif')

Stack dimensions: (1359, 2048, 2048)


In [88]:
# Convert image stack to folder containing individual images.
image_stack = r'/data/Lucas/AL207/AL207_rLS.tif'
output_folder = r'/data/Lucas/AL207/AL207_1'
mice_name = r'AL207_rLS'
from_stack_to_folder(image_stack, output_folder, mice_name, '.tif')

Stack dimensions: (1457, 2048, 2048)


In [6]:
# Convert image stack to folder containing individual images.
image_stack = r'/data/Lucas/AL210/AL210_rLS.tif'
output_folder = r'/data/Lucas/AL210/AL210_1'
mouse_name = r'AL210_rLS'
from_stack_to_folder(image_stack, output_folder, mouse_name, '.tif')

Stack dimensions: (1381, 2048, 2048)


In [4]:
# Convert image stack to folder containing individual images.
image_stack = r'/data/Lucas/AL246/AL246_rLS.tif'
output_folder = r'/data/Lucas/AL246/AL246_1'
mouse_name = r'AL246_rLS'
from_stack_to_folder(image_stack, output_folder, mouse_name, '.tif')

Stack dimensions: (1500, 2048, 2048)


In [6]:
image_stack = r'/data/Lucas/AL249/AL249_rLS.tiff'
output_folder = r'/data/Lucas/AL249/AL249_1'
mouse_name = r'AL249_rLS'
from_stack_to_folder(image_stack, output_folder, mouse_name, '.tif')

/data/Lucas/AL249/AL249_1 already exists. Will be overwritten.
Stack dimensions: (1500, 2048, 2048)


In [ ]:
image_stack = r'/data/Lucas/AL257/AL257_561_rLS.tiff'
output_folder = r'/data/Lucas/AL257/AL257_1'
mouse_name = r'AL257_561_rLS'
from_stack_to_folder(image_stack, output_folder, mouse_name, '.tiff')

**Below is deprecated**, you should run the functions from a terminal window. 

- Check batch size, loaded weights, input/output folders before running

In [10]:
# Run trailmap: terminal command, specify: 1)script file, 2) input folder
# Performance depends on batch size
%run /home/lucasdelez/Documents/TRAILMAP/segment_brain_batch.py /home/lucasdelez/Documents/segmented_images/Al207_1

2022-03-21 09:55:53.420332: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1555] Found device 0 with properties: 
pciBusID: 0000:65:00.0 name: NVIDIA GeForce RTX 3090 computeCapability: 8.6
coreClock: 1.86GHz coreCount: 82 deviceMemorySize: 23.70GiB deviceMemoryBandwidth: 871.81GiB/s
2022-03-21 09:55:53.420481: I tensorflow/stream_executor/platform/default/dso_loader.cc:44] Successfully opened dynamic library libcudart.so.10.1
2022-03-21 09:55:53.420504: I tensorflow/stream_executor/platform/default/dso_loader.cc:44] Successfully opened dynamic library libcublas.so.10
2022-03-21 09:55:53.420524: I tensorflow/stream_executor/platform/default/dso_loader.cc:44] Successfully opened dynamic library libcufft.so.10
2022-03-21 09:55:53.420543: I tensorflow/stream_executor/platform/default/dso_loader.cc:44] Successfully opened dynamic library libcurand.so.10
2022-03-21 09:55:53.420562: I tensorflow/stream_executor/platform/default/dso_loader.cc:44] Successfully opened dynamic library libcus

/home/lucasdelez/Documents/segmented_images/seg-Al207_1 already exists. Will be overwritten
Name: Al207_1
[                                        ]   0%       ETA: Pending        

2022-03-21 10:00:42.223684: I tensorflow/stream_executor/platform/default/dso_loader.cc:44] Successfully opened dynamic library libcudnn.so.7
2022-03-21 10:15:04.800179: W tensorflow/stream_executor/gpu/redzone_allocator.cc:312] Not found: ./bin/ptxas not found
Relying on driver to perform ptx compilation. This message will be only logged once.
2022-03-21 10:15:04.911760: I tensorflow/stream_executor/platform/default/dso_loader.cc:44] Successfully opened dynamic library libcublas.so.10


[========================================] 100%       Total: 46.6 mins    



In [ ]:
# To process labels
%run TRAILMAP-master/prepare_data.py "process_labels" E:/Lucas/Training data/Labels

e workstations were operated by Linux Ubunt## 2. Skeletons 

### 2.1 Skeletonize_3d
Utilizes the trailmap output images from Trailmap in order to get a skeletonized stack. <br/>
The skeleton algorithm should work better with high z resolution. <br/><br/>
I tried to follow the algorithm presented in TRAILMAP paper. It uses 8 different thresholds to "segmentate/cut" pixels in function of their probability to be an axon. <br/> and The skeletonize_3d function from skimage.morphology is then used in order to get the skeletons of the axons. Finally, all resulting skeleton images are summed.  

In [4]:
skeletons_3d(r'/data/Lucas/AL207/seg-Al207_dense', r"/data/Lucas/AL207")
#img_stack = skeleton_3d('/home/lucasdelez/Documents/segmented_images/seg-Al207_substack/*.tif', output_folder, True)
#img_stack = skeleton_3d(file_names, output_folder, True)

Skeleton for threshold 0.2 computed !
/data/Lucas/AL207/skel-seg-Al207_dense/skel_thresh_0 directory created !
Images for threshold 0.2 saved !
Skeleton for threshold 0.30000000000000004 computed !
/data/Lucas/AL207/skel-seg-Al207_dense/skel_thresh_1 directory created !
Images for threshold 0.30000000000000004 saved !
Skeleton for threshold 0.4 computed !
/data/Lucas/AL207/skel-seg-Al207_dense/skel_thresh_2 directory created !
Images for threshold 0.4 saved !
Skeleton for threshold 0.5 computed !
/data/Lucas/AL207/skel-seg-Al207_dense/skel_thresh_3 directory created !
Images for threshold 0.5 saved !
Skeleton for threshold 0.6000000000000001 computed !
/data/Lucas/AL207/skel-seg-Al207_dense/skel_thresh_4 directory created !
Images for threshold 0.6000000000000001 saved !
Skeleton for threshold 0.7000000000000001 computed !
/data/Lucas/AL207/skel-seg-Al207_dense/skel_thresh_5 directory created !
Images for threshold 0.7000000000000001 saved !
Skeleton for threshold 0.8 computed !
/data/

['/data/Lucas/AL207/skel-seg-Al207_dense/skel_thresh_0',
 '/data/Lucas/AL207/skel-seg-Al207_dense/skel_thresh_1',
 '/data/Lucas/AL207/skel-seg-Al207_dense/skel_thresh_2',
 '/data/Lucas/AL207/skel-seg-Al207_dense/skel_thresh_3',
 '/data/Lucas/AL207/skel-seg-Al207_dense/skel_thresh_4',
 '/data/Lucas/AL207/skel-seg-Al207_dense/skel_thresh_5',
 '/data/Lucas/AL207/skel-seg-Al207_dense/skel_thresh_6',
 '/data/Lucas/AL207/skel-seg-Al207_dense/skel_thresh_7']

In [3]:
skeletons_3d(r'/data/Lucas/AL211/seg-AL211_20220506_2', r'/data/Lucas/AL211')
skeletons_3d(r'/data/Lucas/AL213/seg-AL213_20220506_2', r'/data/Lucas/AL213')
skeletons_3d(r'/data/Lucas/AL209/seg-AL209_20220506_2', r'/data/Lucas/AL209')

Skeleton for threshold 0.2 computed !
/data/Lucas/AL211/skel-seg-AL211_20220506_2/skel_thresh_0 directory created !
Images for threshold 0.2 saved !
Skeleton for threshold 0.30000000000000004 computed !
/data/Lucas/AL211/skel-seg-AL211_20220506_2/skel_thresh_1 directory created !
Images for threshold 0.30000000000000004 saved !
Skeleton for threshold 0.4 computed !
/data/Lucas/AL211/skel-seg-AL211_20220506_2/skel_thresh_2 directory created !
Images for threshold 0.4 saved !
Skeleton for threshold 0.5 computed !
/data/Lucas/AL211/skel-seg-AL211_20220506_2/skel_thresh_3 directory created !
Images for threshold 0.5 saved !
Skeleton for threshold 0.6000000000000001 computed !
/data/Lucas/AL211/skel-seg-AL211_20220506_2/skel_thresh_4 directory created !
Images for threshold 0.6000000000000001 saved !
Skeleton for threshold 0.7000000000000001 computed !
/data/Lucas/AL211/skel-seg-AL211_20220506_2/skel_thresh_5 directory created !
Images for threshold 0.7000000000000001 saved !
Skeleton for t

'/data/Lucas/AL209/skel-seg-AL209_20220506_2/weighted_sum_skeleton'

In [4]:
skeletons_3d_bis(r'/data/Lucas/AL246/seg-AL246_20220506_2', r'/data/Lucas/AL246')

Skeleton for threshold 0.2 computed !
/data/Lucas/AL246/skel-seg-AL246_20220506_2/skel_thresh_0 directory created !
Images for threshold 0.2 saved !
Skeleton for threshold 0.3 computed !
/data/Lucas/AL246/skel-seg-AL246_20220506_2/skel_thresh_1 directory created !
Images for threshold 0.3 saved !
Skeleton for threshold 0.4 computed !
/data/Lucas/AL246/skel-seg-AL246_20220506_2/skel_thresh_2 directory created !
Images for threshold 0.4 saved !
Skeleton for threshold 0.5 computed !
/data/Lucas/AL246/skel-seg-AL246_20220506_2/skel_thresh_3 directory created !
Images for threshold 0.5 saved !
Skeleton for threshold 0.6 computed !
/data/Lucas/AL246/skel-seg-AL246_20220506_2/skel_thresh_4 directory created !
Images for threshold 0.6 saved !
Skeleton for threshold 0.7 computed !
/data/Lucas/AL246/skel-seg-AL246_20220506_2/skel_thresh_5 directory created !
Images for threshold 0.7 saved !
Skeleton for threshold 0.8 computed !
/data/Lucas/AL246/skel-seg-AL246_20220506_2/skel_thresh_6 directory 

'/data/Lucas/AL246/skel-seg-AL246_20220506_2/weighted_sum_skeleton'

In [3]:
skeletons_3d_bis(r'/data/Lucas/AL207/seg-AL207_dense', r'/data/Lucas/AL207')

Skeleton for threshold 0.2 computed !
/data/Lucas/AL207/skel-seg-AL207_dense/skel_thresh_0 directory created !
Images for threshold 0.2 saved !
Skeleton for threshold 0.3 computed !
/data/Lucas/AL207/skel-seg-AL207_dense/skel_thresh_1 directory created !
Images for threshold 0.3 saved !
Skeleton for threshold 0.4 computed !
/data/Lucas/AL207/skel-seg-AL207_dense/skel_thresh_2 directory created !
Images for threshold 0.4 saved !
Skeleton for threshold 0.5 computed !
/data/Lucas/AL207/skel-seg-AL207_dense/skel_thresh_3 directory created !
Images for threshold 0.5 saved !
Skeleton for threshold 0.6 computed !
/data/Lucas/AL207/skel-seg-AL207_dense/skel_thresh_4 directory created !
Images for threshold 0.6 saved !
Skeleton for threshold 0.7 computed !
/data/Lucas/AL207/skel-seg-AL207_dense/skel_thresh_5 directory created !
Images for threshold 0.7 saved !
Skeleton for threshold 0.8 computed !
/data/Lucas/AL207/skel-seg-AL207_dense/skel_thresh_6 directory created !
Images for threshold 0.8 

'/data/Lucas/AL207/skel-seg-AL207_dense/weighted_sum_skeleton'

### 2.2. Removal of small objects

If skeletonize_3d works accordingly. It should be possible to measure the length in 3d of the objects.

Two steps proposed: 
    1. Based on size of labelled objects. 
    2. Based on avg probability of the axon.
    
Two thresholds required. 

Looking at the probability distribution of the axons. Can't really be done at this stage. 
It is necessary to consider what is happening in later stages. 

In [5]:
%load_ext Cython

In [6]:
%%cython
import numpy as np
cimport numpy as np

def compute_sum(np.ndarray[float, ndim=3] vol, np.ndarray[long long, ndim=2] list_coords):
    cdef double sum_proba = 0.0
    cdef int i = 0
    for i in range(len(list_coords)):
        sum_proba += vol[list_coords[i][0],list_coords[i][1],list_coords[i][2]]
    
    return sum_proba/len(list_coords)

In [7]:
def compute_skel_probas(stack):
    # Look at the probability distributions of data within the stacks
    # Compute the means but should look into the distribution
    
    label_stack = np.copy(stack) # forces a copy
    if len(label_stack[label_stack < 0]) > 0:
        print('Error: values below 0 detected within the .tif stack')
        print('Stack exited with no change')
        return stack
    label_stack[label_stack > 0] = 1
    label_stack = measure.label(label_stack, background=0)
    print('In stack: individual "blobs":', np.max(label_stack)) #Sanity check
    
    propsa = measure.regionprops(label_stack)
    probablity_densities = []
    for count, item in enumerate(propsa):
        probablity_densities.append(compute_sum(stack, item.coords))
                
    return probablity_densities

In [8]:
stack = from_folder_to_stack(r'/data/Lucas/AL207/skel-seg-Al207_dense/weighted_sum_skeleton')
stack = trim_small_elements(stack, 1000, check=True)

Start: individual "blobs": 108936
Blobs that are removed: 108664


In [ ]:
probability_densities = compute_skel_probas(stack)

In [8]:
print(probability_densities)

[1.1905657812054988, 0.975129300655756, 1.2805225802076685, 0.9739292485420931, 0.9993464176923065, 1.2093767674701525, 0.6472422122883854, 0.8386904836055779, 0.7927576704443663, 0.8079825921094989, 0.6864784620979034, 0.8591528070270564, 0.7835987367352862, 1.33976111250831, 1.1195369158290576, 0.8437801437271021, 1.2527542506676104, 1.4052197980479553, 0.9171854530754266, 0.9445876361821423, 1.3154135515871352, 0.6996800069570541, 1.1405154767263797, 1.0583650301362852, 0.9410635257797368, 0.6992366504464441, 0.6792869358141678, 0.9332988720229272, 0.7236972774561522, 0.8760330678510272, 1.1017361222421405, 1.1278180084128662, 0.683361928724711, 1.0763135371949526, 1.1591800476778, 0.8703508872996297, 0.8353139106267771, 1.0493372723745493, 0.6157963517129266, 0.8080173425723517, 1.0163461653133616, 0.9220149345807175, 1.1020512914619385, 1.2937390437472835, 1.0782758759270454, 1.122657653367685, 0.8668402884052031, 0.7156051046387025, 1.3845454694466157, 1.015587108908161, 0.712656

In [9]:
print(np.min(probability_densities), np.max(probability_densities))

0.5213592262233345 3.9329843725521525


In [11]:
from_vol_to_folder(stack, r'weighted_skeleton.tif', r'/data/Lucas/AL207')

Stack dimensions: (1457, 2048, 2048)


In [4]:
folder_list = get_dir(r'/data/Lucas/AL215/skel-seg-AL215_20220506_2')[:-1]
print(folder_list)

['/data/Lucas/AL215/skel-seg-AL215_20220506_2/skel_thresh_0', '/data/Lucas/AL215/skel-seg-AL215_20220506_2/skel_thresh_1', '/data/Lucas/AL215/skel-seg-AL215_20220506_2/skel_thresh_2', '/data/Lucas/AL215/skel-seg-AL215_20220506_2/skel_thresh_3', '/data/Lucas/AL215/skel-seg-AL215_20220506_2/skel_thresh_4', '/data/Lucas/AL215/skel-seg-AL215_20220506_2/skel_thresh_5', '/data/Lucas/AL215/skel-seg-AL215_20220506_2/skel_thresh_6', '/data/Lucas/AL215/skel-seg-AL215_20220506_2/skel_thresh_7']


In [6]:
#skeletons_3d_bis(r'/data/Lucas/AL215/seg-AL215_20220506_2', r"/data/Lucas/AL215")
thresh_list = np.array(0.1*np.array(range(2,10,1))).astype('float32')
weighted_sum_skeleton(folder_list, thresh_list, r'/data/Lucas/AL215/skel-seg-AL215_20220506_2')

In [29]:
folder_list = get_dir(r'/data/Lucas/AL210/skel-seg-AL210_20220428_4')[:-1]
print(folder_list)
thresh_list = 0.1*np.array(range(2,10,1))
weighted_sum_skeleton(folder_list, thresh_list, r'/data/Lucas/AL210/skel-seg-AL210_20220428_4')

['/data/Lucas/AL210/skel-seg-AL210_20220428_4/skel_thresh_0', '/data/Lucas/AL210/skel-seg-AL210_20220428_4/skel_thresh_1', '/data/Lucas/AL210/skel-seg-AL210_20220428_4/skel_thresh_2', '/data/Lucas/AL210/skel-seg-AL210_20220428_4/skel_thresh_3', '/data/Lucas/AL210/skel-seg-AL210_20220428_4/skel_thresh_4', '/data/Lucas/AL210/skel-seg-AL210_20220428_4/skel_thresh_5', '/data/Lucas/AL210/skel-seg-AL210_20220428_4/skel_thresh_6', '/data/Lucas/AL210/skel-seg-AL210_20220428_4/skel_thresh_7']


In [8]:
%load_ext Cython

In [9]:
%%cython
import numpy as np
cimport numpy as np

def set_to_zero(np.ndarray[float, ndim=3] vol, np.ndarray[long long, ndim=2] list_coords):
    cdef int i = 0
    for i in range(len(list_coords)):
        vol[list_coords[i][0],list_coords[i][1],list_coords[i][2]] = 0
    
    return vol

In [10]:
def trim_small_elements(stack_filename, min_threshold, max_threshold=-1):
    # Removing small elements increases the mean probability of remaining pixels. (Post-skeletonization)
    # Small elements should be associated with lower probabilities of axon presence (artifacts, blood vessels,...)
    # We want the "big" stuff
    
    label_stack = from_folder_to_stack(stack_filename) # loading stack a first time for connected-components labeling
    label_stack = label_stack.view(dtype=np.float32) # forces a specific size (should be native size after segmentation) (check if worth doing)
    label_stack[label_stack > 0] = 1 # binarization, checking below 0 should be unnecessary (if proper workflow order)
    label_stack, N = cc3d.connected_components(label_stack, out_dtype=np.uint32, return_N=True) # better than skimage ?
    #label_stack = measure.label(label_stack, background=0) #computationally expensive
    print('Start: individual "blobs":', N)
    
    propsa = measure.regionprops(label_stack)
    del label_stack #liberate memory
    
    index_list = []
    for count, item in enumerate(propsa):
        if len(item.coords) < min_threshold:
            index_list.append(count)
        elif max_threshold > 0 and len(item.coords) > max_threshold:
            index_list.append(count)
    
    #del propsa #liberate memory
    print('Blobs that are removed:', len(index_list))
    
    stack = from_folder_to_stack(stack_filename) # loading stack a second time
    stack = stack.view(dtype=np.float32) # Need to check if it is worth doing
    
    for i in range(len(index_list)):
        stack = set_to_zero(stack, propsa[index_list[i]].coords)
    
    print('End: individual "blobs" remaining', N-len(index_list))
                
    return stack

In [16]:
i_list, prop = find_biggest_elements(r'/data/Lucas/AL207/skel-seg-AL207_20220506_2/weighted_sum_skeleton', 10000)

Individual "blobs": 178285
Blobs of size superior to 10000: 14


In [18]:
print_biggest_connected_components(i_list, prop)

1110 3652024
5089 44222
8335 12154
18790 14606
20044 49931
23984 11978
82169 32360
83584 10596
85569 32062
99631 13818
109569 22464
137546 12628
138428 14537
158562 28891


In [13]:
stack = trim_small_elements(r'/data/Lucas/AL207/skel-seg-AL207_20220506_2/weighted_sum_skeleton', 1e6)
from_vol_to_folder(stack, r'weighted_skeleton.tif', r'Al207_20220506_2_skel_trim_1e6')
del stack

Start: individual "blobs": 178285
Blobs that are removed: 178284
End: individual "blobs" remaining 1
New directory created at: Al207_20220506_2_skel_trim_1e6
Stack dimensions: (1457, 2048, 2048)
Al207_20220506_2_skel_trim_1e6 already exists. Will be overwritten.


In [19]:
stack = trim_small_elements(r'/data/Lucas/AL207/skel-seg-AL207_20220506_2/weighted_sum_skeleton', 1e4)
from_vol_to_folder(stack, r'weighted_skeleton.tif', r'Al207_20220506_2_skel_trim_1e4')
del stack

ERROR! Session/line number was not unique in database. History logging moved to new session 240
Start: individual "blobs": 178285
Blobs that are removed: 178271
End: individual "blobs" remaining 14
New directory created at: Al207_20220506_2_skel_trim_1e4
Stack dimensions: (1457, 2048, 2048)
Al207_20220506_2_skel_trim_1e4 already exists. Will be overwritten.


In [20]:
i_list, prop = find_biggest_elements(r'/data/Lucas/AL215/skel-seg-AL215_20220506_2/weighted_sum_skeleton', 10000)

Individual "blobs": 250430
Blobs of size superior to 10000: 11


In [21]:
print_biggest_connected_components(i_list, prop)

6 19632984
3766 11522
13631 25157
21192 2732424
26906 26341
124828 12278
156659 67780
157238 13845
186140 10015
211523 12148
215752 36056


In [22]:
stack = trim_small_elements(r'/data/Lucas/AL215/skel-seg-AL215_20220506_2/weighted_sum_skeleton', 1e4)
from_vol_to_folder(stack, r'weighted_skeleton.tif', r'Al215_20220506_2_skel_trim_1e4')
del stack

Start: individual "blobs": 250430
Blobs that are removed: 250419
End: individual "blobs" remaining 11
New directory created at: Al215_20220506_2_skel_trim_1e4
Stack dimensions: (1359, 2048, 2048)
Al215_20220506_2_skel_trim_1e4 already exists. Will be overwritten.


In [23]:
stack = trim_small_elements(r'/data/Lucas/AL215/skel-seg-AL215_20220506_2/weighted_sum_skeleton', 1e4)
from_vol_to_folder(stack, r'weighted_skeleton.tif', r'Al215_20220506_2_skel_trim_1e6')
del stack

Start: individual "blobs": 250430
Blobs that are removed: 250419
End: individual "blobs" remaining 11
New directory created at: Al215_20220506_2_skel_trim_1e6
Stack dimensions: (1359, 2048, 2048)
Al215_20220506_2_skel_trim_1e6 already exists. Will be overwritten.


In [29]:
stack = trim_small_elements(r'/data/Lucas/AL257/skel-seg-AL257_20220506_2/weighted_sum_skeleton', 1e4)
from_vol_to_folder(stack, r'weighted_skeleton.tif', r'/data/Lucas/AL257/AL257_20220506_2_skel_trim_1e4')
del stack

Start: individual "blobs": 315179
Blobs that are removed: 315175
End: individual "blobs" remaining 4
New directory created at: /data/Lucas/AL257/AL257_20220506_2_skel_trim_1e4
Stack dimensions: (1296, 2048, 2048)
/data/Lucas/AL257/AL257_20220506_2_skel_trim_1e4 already exists. Will be overwritten.


In [30]:
stack = trim_small_elements(r'/data/Lucas/AL210/skel-seg-AL210_20220506_2/weighted_sum_skeleton', 1e4)
from_vol_to_folder(stack, r'weighted_skeleton.tif', r'/data/Lucas/AL210/AL210_20220506_2_skel_trim_1e4')
del stack

Start: individual "blobs": 188572
Blobs that are removed: 188566
End: individual "blobs" remaining 6
New directory created at: r"/data/Lucas/AL210/AL210_20220506_2_skel_trim_1e4
Stack dimensions: (1381, 2048, 2048)
r"/data/Lucas/AL210/AL210_20220506_2_skel_trim_1e4 already exists. Will be overwritten.


## 3. Corrections and Atlas alignment

### 3.1 Reads segmentation output and create a .txt of coordinates

Writes (.txt) files from segmentation results. Modified from original code. 

In [9]:
from Neuron_analysis import *
from anne_module import *

In [4]:
mouse = 'AL207'
coords = return_3d_coordinates_from_seg(f'/data/Lucas/{mouse}/{mouse}_20220506_2_skel_fine_trim')
write_atlas_file_from_segdf(coords, f'/data/Lucas/Elastix/downsampled_data', 5 , 25, f"{mouse}_fine")

saved at /data/Lucas/Elastix/downsampled_data/AL207_fine_25um_points.txt


In [5]:
mouse = 'AL210'
coords = return_3d_coordinates_from_seg(f'/data/Lucas/{mouse}/{mouse}_20220506_2_skel_fine_trim')
write_atlas_file_from_segdf(coords, f'/data/Lucas/Elastix/downsampled_data', 5 , 25, f"{mouse}_fine")

saved at /data/Lucas/Elastix/downsampled_data/AL210_fine_25um_points.txt


In [6]:
mouse = 'AL215'
coords = return_3d_coordinates_from_seg(f'/data/Lucas/{mouse}/{mouse}_20220506_2_skel_fine_trim')
write_atlas_file_from_segdf(coords, f'/data/Lucas/Elastix/downsampled_data', 5 , 25, f"{mouse}_fine")

saved at /data/Lucas/Elastix/downsampled_data/AL215_fine_25um_points.txt


In [10]:
mouse = 'AL257'
coords = return_3d_coordinates_from_seg(f'/data/Lucas/{mouse}/{mouse}_20220506_2_skel_trim_1e6')
write_atlas_file_from_segdf(coords, f'/data/Lucas/Elastix/downsampled_data', 5 , 25, f"{mouse}_fine")

saved at /data/Lucas/Elastix/downsampled_data/AL257_fine_25um_points.txt


In [8]:
mouse = 'AL213'
coords = return_3d_coordinates_from_seg(f'/data/Lucas/{mouse}/{mouse}_20220506_2_skel_trim_1e6')
write_atlas_file_from_segdf(coords, f'/data/Lucas/Elastix/downsampled_data', 5 , 25, f"{mouse}_fine")

saved at /data/Lucas/Elastix/downsampled_data/AL213_fine_25um_points.txt


### 3.2 Elastix and Transformix to adjust atlas

Atlas alignment using Elastix.
Can be done using the terminal or by running simple Elastix
- First part is to check the Allen template file
- Second part is to downsample the original brain to template dimensions
- You need to crop the stack in order to facilitate the registration
- You can then perform registration using the downsampled autofluorescence brain as moving img and the cropped reference atlas as fixed image

In [4]:
from skimage.measure import block_reduce
from datetime import datetime
from scipy import ndimage

In [7]:
"""
input_folder = r'/data/Lucas/AL211/AL211_1'
output_folder = r'/home/lucasdelez/Documents/AllenBrainCCFv3'

def downscale_imgs_allen(in_folder, out_folder, factor):
    img_stack = dask_image.imread.imread(in_folder + "/*.tif").compute()
    return ndimage.zoom(img_stack, zoom=factor)

zoom_seq = (5/25,5.3/25,5.3/25)
img_stack = downscale_imgs_allen(input_folder, output_folder, factor=zoom_seq) #Factor defined for Mesospim
write_tiff_stack(img_stack, r'/home/lucasdelez/Documents/AllenBrainCCFv3/Al211_25um_corrected.tif')

def downscale_imgs_allen(in_folder, out_folder, factor=(5/25,5.3/25,5.3/25), output_filename):
    img_stack = dask_image.imread.imread(in_folder + "/*.tif").compute()
    img_stack = ndimage.zoom(img_stack, zoom=factor)
    print(f'Saving downscaled stack at {output_filename}...')
    write_tiff_stack(img_stack, output_filename)
    print(f'Saving done !')
    return img_stack
"""

input_folder = r'/data/Lucas/AL257/AL257_1'
output_folder = r'/home/lucasdelez/Documents/AllenBrainCCFv3'

def downscale_imgs_allen(in_folder, out_folder, factor):
    img_stack = dask_image.imread.imread(in_folder + "/*.tiff").compute()
    return block_reduce(img_stack, block_size=factor, func=np.mean)

img_stack = downscale_imgs_allen(input_folder, output_folder, factor=(5,5,5)) #Factor defined for Mesospim
write_tiff_stack(img_stack, r'/home/lucasdelez/Documents/AllenBrainCCFv3/AL257_25um.tif')

In [ ]:
input_folder = r'/data/Lucas/AL211/AL211_1'
output_folder = r'/home/lucasdelez/Documents/AllenBrainCCFv3'

def downscale_imgs_allen(in_folder, out_folder, factor):
    img_stack = dask_image.imread.imread(in_folder + "/*.tif").compute()
    return ndimage.zoom(img_stack, zoom=factor)

zoom_seq = (5/25,5.3/25,5.3/25)
img_stack = downscale_imgs_allen(input_folder, output_folder, factor=zoom_seq) #Factor defined for Mesospim
write_tiff_stack(img_stack, r'/home/lucasdelez/Documents/AllenBrainCCFv3/Al211_25um_corrected.tif')

In [4]:
# Call registration method

# To change
output_directory = r"/home/lucasdelez/Documents/AllenBrainCCFv3/Registration_AL211"
moving_img_autof = r'/home/lucasdelez/Documents/AllenBrainCCFv3/Al211_25um_corrected.tif'


# To keep
fixed_img_cropped_refatlas = r"/home/lucasdelez/Documents/AllenBrainCCFv3/template_65.tif"
param1 = r'/home/lucasdelez/Documents/master_project/elastix_params/clearmap_params/align_affine.txt'
param2 = r'/home/lucasdelez/Documents/master_project/elastix_params/clearmap_params/align_bspline.txt'
Registration(output_directory, fixed_img_cropped_refatlas, moving_img_autof, param1, param2)

Elastix done !


In [ ]:
tfx_output_directory = output_directory + r'/Transformation'
t0_filename = output_directory + r'/TransformParameters.1.txt'
points_directory = r'/home/lucasdelez/Documents/master_project/data/'

Transformation(tempdir, fixed_img_cropped_refatlas, moving_img_autof, param1, param2, t0_filename, points_filename)

In [3]:
def compute_transformation(output_directory, moving_img_autof, points_filename):
    fixed_img_cropped_refatlas = r"/home/lucasdelez/Documents/AllenBrainCCFv3/template_65.tif"
    param1 = r'/home/lucasdelez/Documents/master_project/elastix_params/clearmap_params/align_affine.txt'
    param2 = r'/home/lucasdelez/Documents/master_project/elastix_params/clearmap_params/align_bspline.txt'
    Registration(output_directory, fixed_img_cropped_refatlas, moving_img_autof, param1, param2)
    
    tfx_output_directory = output_directory + r'/Transformation'
    if not os.path.exists(tfx_output_directory):
        os.makedirs(tfx_output_directory)
        print(tfx_output_directory, "directory created !")
    t0_filename = output_directory + r'/TransformParameters.1.txt'
    Transformation(tfx_output_directory, fixed_img_cropped_refatlas, moving_img_autof, param1, param2, t0_filename, points_filename)
    
def compute_transformation_wo_registration(output_directory, moving_img_autof, points_filename):
    fixed_img_cropped_refatlas = r"/home/lucasdelez/Documents/AllenBrainCCFv3/template_65.tif"
    param1 = r'/home/lucasdelez/Documents/master_project/elastix_params/clearmap_params/align_affine.txt'
    param2 = r'/home/lucasdelez/Documents/master_project/elastix_params/clearmap_params/align_bspline.txt'
    
    tfx_output_directory = output_directory + r'/Transformation'
    if not os.path.exists(tfx_output_directory):
        os.makedirs(tfx_output_directory)
        print(tfx_output_directory, "directory created !")
    t0_filename = output_directory + r'/TransformParameters.1.txt'
    Transformation(tfx_output_directory, fixed_img_cropped_refatlas, moving_img_autof, param1, param2, t0_filename, points_filename)
    
    
def apply_transformation(output_directory, moving_img_autof, points_filename):
    """
    Not tested.
    Assumes that all the directories exists.
    Need to set "InitialTransformParametersFileName" in TransformParameters.0.txt to no "NoInitialTransform"
    """
    tfx_output_directory = output_directory + r'/Transformation'
    parameter_filename_0 = tfx_output_directory + r'TransformParameters.0.txt'
    parameter_filename_1 = tfx_output_directory + r'TransformParameters.1.txt'
    Transformation_noInverse(tfx_output_directory, points_filename, moving_img_autof, parameter_filename_0, parameter_filename_1)
    

In [10]:
#compute_transformation(r'/data/Lucas/Elastix/AL207', r'/home/lucasdelez/Documents/AllenBrainCCFv3/Al207_25um.tif', r'/home/lucasdelez/Documents/master_project/data/AL207_e4_25um_points.txt')
#compute_transformation(r'/data/Lucas/Elastix/AL210', r'/home/lucasdelez/Documents/AllenBrainCCFv3/Al210_25um.tif', r'/home/lucasdelez/Documents/master_project/data/AL210_25um_points_1e4.txt')
#compute_transformation_wo_registration(r'/data/Lucas/Elastix/AL207', r'/home/lucasdelez/Documents/AllenBrainCCFv3/Al207_25um.tif', r'/home/lucasdelez/Documents/master_project/data/AL207_e4_25um_points.txt')
#compute_transformation(r'/data/Lucas/Elastix/AL210', r'/home/lucasdelez/Documents/AllenBrainCCFv3/Al210_25um.tif', r'/home/lucasdelez/Documents/master_project/data/AL210_25um_points_1e4.txt')
#compute_transformation_wo_registration(r'/data/Lucas/Elastix/AL210', r'/home/lucasdelez/Documents/AllenBrainCCFv3/Al210_25um.tif', r'/home/lucasdelez/Documents/master_project/data/AL210_e6_25um_points.txt')
compute_transformation(r'/data/Lucas/Elastix/AL209', r'/home/lucasdelez/Documents/AllenBrainCCFv3/Al213_25um.tif', r'/home/lucasdelez/Documents/master_project/data/AL209_e6_25um_points.txt')
compute_transformation(r'/data/Lucas/Elastix/AL213', r'/home/lucasdelez/Documents/AllenBrainCCFv3/Al213_25um.tif', r'/home/lucasdelez/Documents/master_project/data/AL213_e6_25um_points.txt')

Elastix done !
/data/Lucas/Elastix/AL213/Transformation directory created !
Transformix done !


In [ ]:
skeletons_3d(r'/data/Lucas/AL211/seg-AL211_20220506_2', r'/data/Lucas/AL211')
skeletons_3d(r'/data/Lucas/AL213/seg-AL213_20220506_2', r'/data/Lucas/AL213')

In [7]:
compute_transformation_wo_registration(r'/data/Lucas/Elastix/AL207', r'/home/lucasdelez/Documents/AllenBrainCCFv3/Al207_25um.tif', r'/data/Lucas/Elastix/downsampled_data/AL207_fine_25um_points.txt')
compute_transformation_wo_registration(r'/data/Lucas/Elastix/AL210', r'/home/lucasdelez/Documents/AllenBrainCCFv3/Al210_25um.tif', r'/data/Lucas/Elastix/downsampled_data/AL210_fine_25um_points.txt')
compute_transformation(r'/data/Lucas/Elastix/AL215', r'/home/lucasdelez/Documents/AllenBrainCCFv3/Al215_25um.tif', r'/data/Lucas/Elastix/downsampled_data/AL215_fine_25um_points.txt')

Transformix done !
Transformix done !
Elastix done !
/data/Lucas/Elastix/AL215/Transformation directory created !
Transformix done !


In [12]:
#mouse='AL213'
#compute_transformation(f'/data/Lucas/Elastix/{mouse}', f'/home/lucasdelez/Documents/AllenBrainCCFv3/{mouse}_25um.tif', f'/data/Lucas/Elastix/downsampled_data/{mouse}_fine_25um_points.txt')
mouse='AL257'
compute_transformation_wo_registration(f'/data/Lucas/Elastix/{mouse}', f'/home/lucasdelez/Documents/AllenBrainCCFv3/{mouse}_25um.tif', f'/data/Lucas/Elastix/downsampled_data/{mouse}_fine_25um_points.txt')

Transformix done !


### 3.3 Extract and correct atlas coordinates ???

I can't test this part yet.

It is necessary to downsample our brain to 25um x 25um x 25um in order to match the Allen atlas.
Registration is performed via Elastix and Transformix (alignment btw atlas brain and our brains).

**get_pt_natlas** reads (.txt) files from precedent part and eliminates negative values.  

#### >

Relabel some regions of the atlas

## 4. Plots and Results

In [ ]:
region_with_counts= region_counts (points_in_atlas)
sorted_pd=sort_by_parent(region_with_counts,out_name)
plot_hist(sorted_pd,'AL211')#plot as histogram and save as .html

In [ ]:
def parent_df(df):
    ''' group data frame items by parent id structure
    '''
    grouped_pd=df.groupby(['parent_structure_id'],as_index=False).sum()
    d= {'id': grouped_pd.parent_structure_id.astype(int), 'Total_counts': grouped_pd.Total_counts}
    grouped_pd2= pd.DataFrame(data=d)
    result = pd.merge(grouped_pd2, na.atlas_labels, on=["id"])
    result.sort_values(['Total_counts'], ascending=True, inplace=True)
    # result is the final pd
    return result

# group item based on parents and save as excel
result= parent_df(sorted_pd)
result.to_excel(f'{out_name}_parent.xls')  Legacy

## 5. Appendix

In [7]:
input_folder = fdialog.askdirectory(initialdir=os.path.realpath("__file__"), title='Please select the directory containing segmentation image sequences')
output_folder = fdialog.askdirectory(initialdir=os.path.realpath("__file__"), title='Please select the output directory')

downsampling_factor = 10

In [ ]:
def downscale_imgs(in_folder, out_folder, factor):
    images=os.listdir(in_folder)
    images=[i for i in images if i.endswith('.tif')]
    images.sort()
    
    for count,item in enumerate(tqdm(images)):
        if count > -1: #so you don't have to start from beginning if problem
            img = cv2.imread(in_folder + "/" + item, -1)
            out_img = block_reduce(img, block_size=(factor, factor), func=np.mean)
            out_img = Image.fromarray(out_img)
            out_img.save(out_folder + "//" + "ds_" + os.path.basename(item))

In [ ]:
downscale_imgs(input_folder, output_folder, downsampling_factor) 